## Different ways to represent tensors

In [2]:
import tensorflow as tf
import numpy as np

m1 = [[1.0, 2.0],
     [3.0, 4.0]]
m2 = np.array([[1.0, 2.0],
              [3.0, 4.0]], dtype=np.float32)
m3 = tf.constant([[1.0, 2.0],
                 [3.0, 4.0]])

print(type(m1))
print(type(m2))
print(type(m3))

<type 'list'>
<type 'numpy.ndarray'>
<class 'tensorflow.python.framework.ops.Tensor'>


In [29]:
t1 = tf.convert_to_tensor(m1, dtype=tf.float32)
t2 = tf.convert_to_tensor(m2, dtype=tf.float32)
t3 = tf.convert_to_tensor(m3, dtype=tf.float32)

print(type(t1))
print(type(t2))
print(type(t3))

<class 'tensorflow.python.framework.ops.Tensor'>
<class 'tensorflow.python.framework.ops.Tensor'>
<class 'tensorflow.python.framework.ops.Tensor'>


## Creating tensors

In [30]:
matrix1 = tf.constant([[1., 2.]])
matrix2 = tf.constant([[1],[2]])
myTensor = tf.constant([[[1,2],[3,4],[5,6]],
                        [[7,8],[9,10],[11,12]]])
print(matrix1)
print(matrix2)
print(myTensor)                        

Tensor("Const_5:0", shape=(1, 2), dtype=float32)
Tensor("Const_6:0", shape=(2, 1), dtype=int32)
Tensor("Const_7:0", shape=(2, 3, 2), dtype=int32)


## Using the negation operator

In [6]:
x = tf.constant([[1, 2]])
neg_x = tf.neg(x)
print(neg_x)

with tf.Session() as sess:
    result = sess.run(neg_x)
print(result)

Tensor("Neg_1:0", shape=TensorShape([Dimension(1), Dimension(2)]), dtype=int32)
[[-1 -2]]


## Using the interactive session mode

In [9]:
sess = tf.InteractiveSession()
result = neg_x.eval()
print(result)
sess.close()

[[-1 -2]]


## Logging a session

In [10]:
with tf.Session(config=tf.ConfigProto(log_device_placement=True)) as sess:
    result = sess.run(neg_x)
print(result)

[[-1 -2]]


## Using a variable

In [11]:
sess =tf.InteractiveSession()
raw_data = [1., 2., 8., -1., 0., 5.5, 6., 13]
spike = tf.Variable(False)
spike.initializer.run()

for i in range(1, len(raw_data)):
    if raw_data[i] - raw_data[i-1] > 5:
        updater = tf.assign(spike, True)
        updater.eval()
    else:
        tf.assign(spike, False).eval()
    print("Spike", spike.eval())
sess.close()

('Spike', False)
('Spike', True)
('Spike', False)
('Spike', False)
('Spike', True)
('Spike', False)
('Spike', True)


## Saving variables

In [40]:
sess =tf.InteractiveSession()
raw_data = [1., 2., 8., -1., 0., 5.5, 6., 13]
spikes = tf.Variable([False] * len(raw_data), name='spikes')
spikes.initializer.run()

saver = tf.train.Saver({'spikes': spikes})

for i in range(1, len(raw_data)):
    if raw_data[i] - raw_data[i-1] > 5:
        spikes_val = spikes.eval()
        spikes_val[i] = True
        updater = tf.assign(spikes, spikes_val)
        updater.eval()

save_path = saver.save(sess, "spikes.ckpt")
print("spikes data saved in file: %s" % save_path)
sess.close()

spikes data saved in file: spikes.ckpt


## Loading variables

In [43]:
sess = tf.InteractiveSession()
spikes = tf.Variable([False] * len(raw_data), name='spikes')
saver = tf.train.Saver({'spikes':spikes})

saver.restore(sess, "./spikes.ckpt")
print(spikes.eval())
sess.close()

[False False  True False False  True False  True]


## Visualizing data using TensorBoard

In [3]:
raw_data = np.random.normal(10, 1, 100)

alpha = tf.constant(0.05)
curr_value = tf.placeholder(tf.float32)
prev_avg = tf.Variable(0.)
update_avg = alpha * curr_value + (1 - alpha) * prev_avg

avg_hist = tf.scalar_summary("running average", update_avg)
value_hist = tf.scalar_summary("incoming values", curr_value)
merged = tf.merge_all_summaries()
writer = tf.train.SummaryWriter("./logs")

init = tf.initialize_all_variables()

with tf.Session() as sess:
    sess.run(init)
    for i in range(len(raw_data)):
        summary_str, curr_avg = sess.run([merged, update_avg], feed_dict={curr_value:raw_data[i]})
        sess.run(tf.assign(prev_avg, curr_avg))
        print(raw_data[i], curr_avg)
        writer.add_summary(summary_str, i)

Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary op is no longer supported.
Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary op is no longer supported.
Instructions for updating:
Please switch to tf.summary.merge_all.
Instructions for updating:
Please switch to tf.summary.merge.
Instructions for updating:
Please switch to tf.summary.FileWriter. The interface and behavior is the same; this is just a rename.
Instructions for updating:
Use `tf.global_variables_initializer` instead.
(9.6625662